In [ ]:
%run imports.py

/home/knull/miniconda3/envs/auto-seg/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# def calculate_model_complexity(model, input_res):
#     macs, params = get_model_complexity_info(model, input_res, as_strings=False, print_per_layer_stat=False)
#     gflops = macs / 1e9 
#     params_million = params / 1e6 
#     return params_million, gflops

In [3]:
# input_size = (3, 256, 256)
# segformer_variants = ["b0", "b1", "b2", "b3", "b4", "b5"]
# unet_variants = [32, 64, 86, 114, 128, 156]

# for i in range(len(segformer_variants)):
#     segformer = select_model(model_name="segformer", model_config=segformer_variants[i])
#     seg_params, seg_gflops = calculate_model_complexity(segformer, input_size)

#     unet = select_model(model_name="unet", model_config=unet_variants[i])
#     unet_params, unet_gflops = calculate_model_complexity(unet, input_size)
        
#     print(f"Segformer ({segformer_variants[i]})\t Params(M): {seg_params:.2f}\tGFLOPs: {seg_gflops:.2f} | UNet ({unet_variants[i]})\t Params(M): {unet_params:.2f}\tGFLOPs: {unet_gflops:.2f}")



### Ablation Studies

In [4]:
ROOT = str(env_vars["results_root"])
ablation_kfold_path = f"{ROOT}/ablation-studies"

model_name = "segformer"
model_configs = ["b0", "b1", "b2", "b3"]
variants = ["size_256x64", "size_320x80", "size_384x96"]
results_fname = "results.json"

In [5]:
for model_config in model_configs:
    if model_config=="b0" or model_config=="b1":
        variant = str(variants[0])
        kfold_results_path = f'{ablation_kfold_path}/{model_name}_{model_config}.{variant}.{results_fname}'
        with open(f'{kfold_results_path}', 'r') as file:
            kfold_data = json.load(file)

        keys = list(kfold_data.keys())
        
        benchmark_dice_score, benchmark_miou_score = 0.0, 0.0
        for key in keys:
            benchmark_dice_score += kfold_data[key]["test_dice_score"]
            benchmark_miou_score += kfold_data[key]["test_miou_score"]
            
        benchmark_dice_score, benchmark_miou_score = benchmark_dice_score/len(keys), benchmark_miou_score/len(keys)
        benchmark_dice_score, benchmark_miou_score
        print(f"{model_name}-{model_config}, Used Image size: {variant.split('_')[1]} => Dice: {benchmark_dice_score:2f}, mIoU: {benchmark_miou_score:2f}")
    
    else:
        for variant in variants:
            kfold_results_path = f'{ablation_kfold_path}/{model_name}_{model_config}.{variant}.{results_fname}'
            with open(f'{kfold_results_path}', 'r') as file:
                kfold_data = json.load(file)

            keys = list(kfold_data.keys())
            benchmark_dice_score, benchmark_miou_score = 0.0, 0.0
            for key in keys:
                benchmark_dice_score += kfold_data[key]["test_dice_score"]
                benchmark_miou_score += kfold_data[key]["test_miou_score"]
                
            benchmark_dice_score, benchmark_miou_score = benchmark_dice_score/len(keys), benchmark_miou_score/len(keys)
            benchmark_dice_score, benchmark_miou_score
            print(f"{model_name}-{model_config}, Used Image size: {variant.split('_')[1]} => Dice: {benchmark_dice_score:2f}, mIoU: {benchmark_miou_score:2f}")
            


segformer-b0, Used Image size: 256x64 => Dice: 0.820084, mIoU: 0.698333
segformer-b1, Used Image size: 256x64 => Dice: 0.824564, mIoU: 0.703814
segformer-b2, Used Image size: 256x64 => Dice: 0.837433, mIoU: 0.723890
segformer-b2, Used Image size: 320x80 => Dice: 0.840771, mIoU: 0.728543
segformer-b2, Used Image size: 384x96 => Dice: 0.853057, mIoU: 0.747256
segformer-b3, Used Image size: 256x64 => Dice: 0.835593, mIoU: 0.721535
segformer-b3, Used Image size: 320x80 => Dice: 0.844847, mIoU: 0.734612
segformer-b3, Used Image size: 384x96 => Dice: 0.841592, mIoU: 0.730296


### Final Results - Segformer

In [6]:
SEGFORMER_FINAL_RESULTS_ROOT = f"{ROOT}/final-results-with-segformer"
seg_kfold_path = f"{SEGFORMER_FINAL_RESULTS_ROOT}/train_kfold"
seg_semi_auto_path = f"{SEGFORMER_FINAL_RESULTS_ROOT}/train_semi_auto"
save_folder = "outputs/figures"

seg_model_name = "segformer"
seg_model_config = "b2"
seg_variant = "size_384x96"
seg_results_fname = "results.json"

In [7]:
## Getting the benchmarks score from kfold training
benchmark_dice_score, benchmark_miou_score = 0.0, 0.0
kfold_results_path = f'{seg_kfold_path}/{seg_model_name}_{seg_model_config}.{seg_variant}.{seg_results_fname}'

with open(f'{kfold_results_path}', 'r') as file:
    kfold_data = json.load(file)

keys = list(kfold_data.keys())
for key in keys:
    benchmark_dice_score += kfold_data[key]["test_dice_score"]
    benchmark_miou_score += kfold_data[key]["test_miou_score"]
    
benchmark_dice_score, benchmark_miou_score = benchmark_dice_score/len(keys), benchmark_miou_score/len(keys)
benchmark_dice_score, benchmark_miou_score

(0.8530568847289454, 0.7472556513089399)

In [8]:
### Generating Train & Test Results
with open(f"{seg_semi_auto_path}/{seg_model_name}_{seg_model_config}.{seg_variant}.{results_fname}", "r") as file:
    semi_auto_data = json.load(file)
    
split_list, split_label_list = [], []
presas_dice_score_list, postsas_dice_score_list = [], []
presas_miou_score_list, postsas_miou_score_list = [], []
    
for key in semi_auto_data.keys():
    real_data_pct = int(key[15:])
    split_list.append(real_data_pct)
    split_label_list.append(f'{real_data_pct}%+{80-real_data_pct}%')
    presas_dice_score_list.append(semi_auto_data[key]["presas_test_dice"])
    postsas_dice_score_list.append(semi_auto_data[key]["postsas_test_dice"])
    presas_miou_score_list.append(semi_auto_data[key]["presas_test_miou"])
    postsas_miou_score_list.append(semi_auto_data[key]["postsas_test_miou"])
    print(f'Real Mask {real_data_pct}% + Generated mask {80-real_data_pct}% : \t\tPre-SAS Dice: {semi_auto_data[key]["presas_test_dice"]:.5f}\tPost-SAS: {semi_auto_data[key]["postsas_test_dice"]:.5f} \t\tPre-SAS mIoU: {semi_auto_data[key]["presas_test_miou"]:.5f}\tPost-SAS mIoU: {semi_auto_data[key]["postsas_test_miou"]:.5f}')

Real Mask 10% + Generated mask 70% : 		Pre-SAS Dice: 0.66812	Post-SAS: 0.71032 		Pre-SAS mIoU: 0.50882	Post-SAS mIoU: 0.55848
Real Mask 20% + Generated mask 60% : 		Pre-SAS Dice: 0.75879	Post-SAS: 0.80796 		Pre-SAS mIoU: 0.61586	Post-SAS mIoU: 0.68401
Real Mask 30% + Generated mask 50% : 		Pre-SAS Dice: 0.82642	Post-SAS: 0.84594 		Pre-SAS mIoU: 0.70752	Post-SAS mIoU: 0.73653
Real Mask 40% + Generated mask 40% : 		Pre-SAS Dice: 0.83981	Post-SAS: 0.83512 		Pre-SAS mIoU: 0.72660	Post-SAS mIoU: 0.72163
Real Mask 50% + Generated mask 30% : 		Pre-SAS Dice: 0.85200	Post-SAS: 0.86643 		Pre-SAS mIoU: 0.74455	Post-SAS mIoU: 0.76684
Real Mask 60% + Generated mask 20% : 		Pre-SAS Dice: 0.83499	Post-SAS: 0.86143 		Pre-SAS mIoU: 0.71932	Post-SAS mIoU: 0.75918
Real Mask 70% + Generated mask 10% : 		Pre-SAS Dice: 0.87303	Post-SAS: 0.86279 		Pre-SAS mIoU: 0.77768	Post-SAS mIoU: 0.76261


In [9]:
plot_test_results(benchmark_score=benchmark_dice_score,
                  metric_name="Dice",
                  split_list=split_list, 
                  split_label_list=split_label_list,
                  split_score_list_pre=presas_dice_score_list, 
                  split_score_list_post=postsas_dice_score_list, 
                  savefig_name=f"{save_folder}/segformer_test_dice_scores.pdf");

In [10]:
plot_test_results(benchmark_score=benchmark_miou_score,
                  metric_name="mIoU",
                  split_list=split_list, 
                  split_label_list=split_label_list,
                  split_score_list_pre=presas_miou_score_list, 
                  split_score_list_post=postsas_miou_score_list, 
                  savefig_name=f"{save_folder}/segformer_test_miou_scores.pdf");

In [11]:
## Loading the SAS mask dice scores
gen_dataset_path = f'{seg_semi_auto_path}/gen_data_testset'
files = os.listdir(gen_dataset_path)
dice_score_files = [fname for fname in files if fname.startswith("dice_scores_")]

dice_scores = {"presas": {}, 
               "postsas": {}}

for file in dice_score_files:
    if "presas" in file:
        category = "presas"
    elif "postsas" in file:
        category = "postsas"
    else:
        continue
    
    percent = file.split("_")[-1].replace(".pkl", "")

    with open(f"{gen_dataset_path}/{file}", "rb") as f:
        tensor_dice_scores = pickle.load(f)
        
        dice_scores[category][percent] = [score.item() for idx, score in tensor_dice_scores]
        

In [12]:
plot_dice_score_vs_data_percent(data=dice_scores, 
                                model_name="SegFormer", 
                                model_config="B2", 
                                save_path=f'{save_folder}/segformer_dice_score_data_percent_comparison.pdf', 
                                benchmark_dice_score=benchmark_dice_score)

In [13]:
all_entropies_segformer = plot_dice_score_entropy(data=dice_scores, model_name="UNet", model_config="86", save_path=f'{save_folder}/unet_dice_score_entropy.pdf')

for i in range(len(all_entropies_segformer)):
    entropy_str = ""
    if i==0:
        print("pre-SISM")
    else:
        print("post-SISM")
        
    for val in all_entropies_segformer[i]:
        entropy_str += f"& {val:.3f}"
        
    print(entropy_str)
        

pre-SISM
& 1.971& 1.782& 1.523& 1.475& 1.421& 1.506& 1.274
post-SISM
& 1.904& 1.576& 1.416& 1.475& 1.345& 1.279& 1.349


In [14]:
# summary = multi_model_fit(all_entropies_segformer[1], "for_segformer_entropies_postsism", num_data_point=3)

In [15]:
fit_exp_offset(all_entropies_segformer[1], "for_segformer_entropies_postsism", num_data_point=3)

/home/knull/codespace/semi-automatic-segmentation/utils/visualization.py:520: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(exp_offset, X_train, y_train,


(array([1.31398405, 0.72012987, 1.26451081]),
 1.0,
 array([1.33823269, 1.30039044, 1.28197304, 1.27300949]))

##### For Latex results table: SAS performance

In [16]:
# Dice 
presas_result_str = ""
for score in presas_dice_score_list:
    presas_result_str += f"& {score:.4f}"
    
postsas_result_str = ""
for score in postsas_dice_score_list:
    postsas_result_str += f"& {score:.4f}"
    
print(f"Pre-SAS dice: {presas_result_str}")
print(f"Post-SAS dice: {postsas_result_str}")

# mIoU
presas_result_str = ""
for score in presas_miou_score_list:
    presas_result_str += f"& {score:.4f}"
    
postsas_result_str = ""
for score in postsas_miou_score_list:
    postsas_result_str += f"& {score:.4f}"
    


print(f"Pre-SAS miou: {presas_result_str}")
print(f"Post-SAS miou: {postsas_result_str}")

Pre-SAS dice: & 0.6681& 0.7588& 0.8264& 0.8398& 0.8520& 0.8350& 0.8730
Post-SAS dice: & 0.7103& 0.8080& 0.8459& 0.8351& 0.8664& 0.8614& 0.8628
Pre-SAS miou: & 0.5088& 0.6159& 0.7075& 0.7266& 0.7445& 0.7193& 0.7777
Post-SAS miou: & 0.5585& 0.6840& 0.7365& 0.7216& 0.7668& 0.7592& 0.7626


##### For Latex results table: Mask Quality

In [17]:
categories = ['presas', 'postsas']
percentages = ['10%', '20%', '30%', '40%', '50%', '60%', '70%']

unusables, unreliables, moderates, goods = "", "", "", ""

for category in categories:
    unusables_row, unreliables_row, moderates_row, goods_row = "", "", "", ""
    
    for percent in percentages:
        scores = sorted(dice_scores[category][percent])
        
        unusable = sum(1 for score in scores if score == 0.0)
        unreliable = sum(1 for score in scores if 0.0 < score <= 0.5)
        moderate = sum(1 for score in scores if 0.5 < score <= 0.8)
        good = sum(1 for score in scores if score > 0.8)
        
        unusables_row += f"& {unusable/len(scores)*100:.2f}"
        unreliables_row += f"& {unreliable/len(scores)*100:.2f}"
        moderates_row += f"& {moderate/len(scores)*100:.2f}"
        goods_row += f"& {good/len(scores)*100:.2f}"
    

    print(category)
    print("="*80)
    print("Unusables:\t", unusables_row)
    print("Unreliables:\t", unreliables_row)
    print("Moderates:\t", moderates_row)
    print("Goods:\t\t", goods_row)
    print()


presas
Unusables:	 & 4.50& 3.00& 1.00& 1.00& 0.50& 3.00& 1.00
Unreliables:	 & 19.50& 13.00& 8.00& 8.00& 6.50& 3.50& 6.50
Moderates:	 & 32.00& 22.50& 16.00& 16.00& 15.00& 24.00& 11.50
Goods:		 & 44.00& 61.50& 75.00& 75.00& 78.00& 69.50& 81.00

postsas
Unusables:	 & 1.00& 1.50& 0.50& 1.50& 1.00& 0.50& 0.50
Unreliables:	 & 16.50& 8.50& 7.00& 7.50& 4.50& 6.00& 8.00
Moderates:	 & 26.00& 18.50& 14.50& 16.50& 18.00& 11.50& 12.50
Goods:		 & 56.50& 71.50& 78.00& 74.50& 76.50& 82.00& 79.00



In [18]:
## Splitwise-Modelwise Mask Comparison
full_dataset = get_dataset(image_size=(384, 384), mask_size=(96, 96))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = random.choice(full_dataset)
results = get_sas_modelwise_results(data=data,
                                    model_dir=f"{seg_semi_auto_path}/models", 
                                    model_name="segformer", 
                                    model_config="b2", 
                                    device=device)

In [19]:
plot_modelwise_comparison(image_data=data, results=results, savefig_name=f"{save_folder}/segformer_modelwise_mask_quality_viz.pdf");

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.64].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.64].


### Final Results - UNet

In [20]:
UNET_FINAL_RESULTS_ROOT = f"{ROOT}/final-results-with-unet"
unet_kfold_path = f"{UNET_FINAL_RESULTS_ROOT}/train_kfold"
unet_semi_auto_path = f"{UNET_FINAL_RESULTS_ROOT}/train_semi_auto"
save_folder = "outputs/figures"

unet_model_name = "unet"
unet_model_config = "86"
unet_variant = "size_384x384"
unet_results_fname = "results.json"

In [21]:
## Getting the benchmarks score from kfold training
benchmark_dice_score, benchmark_miou_score = 0.0, 0.0
kfold_results_path = f'{unet_kfold_path}/{unet_model_name}_{unet_model_config}.{unet_variant}.{unet_results_fname}'

with open(f'{kfold_results_path}', 'r') as file:
    kfold_data = json.load(file)

keys = list(kfold_data.keys())
for key in keys:
    benchmark_dice_score += kfold_data[key]["test_dice_score"]
    benchmark_miou_score += kfold_data[key]["test_miou_score"]
    
benchmark_dice_score, benchmark_miou_score = benchmark_dice_score/len(keys), benchmark_miou_score/len(keys)
benchmark_dice_score, benchmark_miou_score

(0.8607764594695148, 0.7583793801419875)

In [22]:
### Generating Train & Test Results
with open(f"{unet_semi_auto_path}/{unet_model_name}_{unet_model_config}.{unet_variant}.{unet_results_fname}", "r") as file:
    semi_auto_data = json.load(file)
    
split_list, split_label_list = [], []
presas_dice_score_list, postsas_dice_score_list = [], []
presas_miou_score_list, postsas_miou_score_list = [], []
    
for key in semi_auto_data.keys():
    real_data_pct = int(key[15:])
    split_list.append(real_data_pct)
    split_label_list.append(f'{real_data_pct}%+{80-real_data_pct}%')
    presas_dice_score_list.append(semi_auto_data[key]["presas_test_dice"])
    postsas_dice_score_list.append(semi_auto_data[key]["postsas_test_dice"])
    presas_miou_score_list.append(semi_auto_data[key]["presas_test_miou"])
    postsas_miou_score_list.append(semi_auto_data[key]["postsas_test_miou"])
    print(f'Real Mask {real_data_pct}% + Generated mask {80-real_data_pct}% : \t\tPre-SAS Dice: {semi_auto_data[key]["presas_test_dice"]:.5f}\tPost-SAS: {semi_auto_data[key]["postsas_test_dice"]:.5f} \t\tPre-SAS mIoU: {semi_auto_data[key]["presas_test_miou"]:.5f}\tPost-SAS mIoU: {semi_auto_data[key]["postsas_test_miou"]:.5f}')

Real Mask 10% + Generated mask 70% : 		Pre-SAS Dice: 0.72437	Post-SAS: 0.72660 		Pre-SAS mIoU: 0.57329	Post-SAS mIoU: 0.57562
Real Mask 20% + Generated mask 60% : 		Pre-SAS Dice: 0.78430	Post-SAS: 0.80296 		Pre-SAS mIoU: 0.65131	Post-SAS mIoU: 0.67580
Real Mask 30% + Generated mask 50% : 		Pre-SAS Dice: 0.81159	Post-SAS: 0.82418 		Pre-SAS mIoU: 0.68744	Post-SAS mIoU: 0.70501
Real Mask 40% + Generated mask 40% : 		Pre-SAS Dice: 0.82538	Post-SAS: 0.80888 		Pre-SAS mIoU: 0.70606	Post-SAS mIoU: 0.68377
Real Mask 50% + Generated mask 30% : 		Pre-SAS Dice: 0.83203	Post-SAS: 0.83420 		Pre-SAS mIoU: 0.71563	Post-SAS mIoU: 0.71868
Real Mask 60% + Generated mask 20% : 		Pre-SAS Dice: 0.84306	Post-SAS: 0.84378 		Pre-SAS mIoU: 0.73157	Post-SAS mIoU: 0.73244
Real Mask 70% + Generated mask 10% : 		Pre-SAS Dice: 0.85413	Post-SAS: 0.85353 		Pre-SAS mIoU: 0.74849	Post-SAS mIoU: 0.74697


In [23]:
plot_test_results(benchmark_score=benchmark_dice_score,
                  metric_name="Dice",
                  split_list=split_list, 
                  split_label_list=split_label_list,
                  split_score_list_pre=presas_dice_score_list, 
                  split_score_list_post=postsas_dice_score_list, 
                  savefig_name=f"{save_folder}/unet_test_dice_scores.pdf");

In [24]:
plot_test_results(benchmark_score=benchmark_miou_score,
                  metric_name="mIoU",
                  split_list=split_list, 
                  split_label_list=split_label_list,
                  split_score_list_pre=presas_miou_score_list, 
                  split_score_list_post=postsas_miou_score_list, 
                  savefig_name=f"{save_folder}/unet_test_miou_scores.pdf");

In [25]:
## Loading the SAS mask dice scores
gen_dataset_path = f'{unet_semi_auto_path}/gen_data_testset'
files = os.listdir(gen_dataset_path)
dice_score_files = [fname for fname in files if fname.startswith("dice_scores_")]

dice_scores = {"presas": {}, 
               "postsas": {}}

for file in dice_score_files:
    if "presas" in file:
        category = "presas"
    elif "postsas" in file:
        category = "postsas"
    else:
        continue
    
    percent = file.split("_")[-1].replace(".pkl", "")

    with open(f"{gen_dataset_path}/{file}", "rb") as f:
        tensor_dice_scores = pickle.load(f)
        
        dice_scores[category][percent] = [score.item() for idx, score in tensor_dice_scores]
        

In [26]:
plot_dice_score_vs_data_percent(data=dice_scores, 
                                model_name="UNet", 
                                model_config="86", 
                                save_path=f'{save_folder}/unet_dice_score_data_percent_comparison.pdf', 
                                benchmark_dice_score=benchmark_dice_score)

In [27]:
all_entropies_unet = plot_dice_score_entropy(data=dice_scores, model_name="UNet", model_config="86", save_path=f'{save_folder}/unet_dice_score_entropy.pdf')

for i in range(len(all_entropies_unet)):
    entropy_str = ""

    if i==0:
        print("pre-SISM")
    else:
        print("post-SISM")
        
    for val in all_entropies_unet[i]:
        entropy_str += f"& {val:.3f}"
        
    print(entropy_str)
        

pre-SISM
& 1.854& 1.750& 1.593& 1.531& 1.473& 1.473& 1.404
post-SISM
& 1.879& 1.663& 1.558& 1.574& 1.476& 1.494& 1.430


In [28]:
# summary = multi_model_fit(all_entropies_unet[1], "for_unet_entropies_postsism", num_data_point=3)

In [29]:
fit_exp_offset_dual(entropy_values1=all_entropies_segformer[1], 
                    entropy_values2=all_entropies_unet[1], 
                    save_fname="optimized_training_set_estimation_with_asymptotic_exp_decay", 
                    num_data_point=3)

/home/knull/codespace/semi-automatic-segmentation/utils/visualization.py:581: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(exp_offset, X_train, y_train,
/home/knull/codespace/semi-automatic-segmentation/utils/visualization.py:581: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(exp_offset, X_train, y_train,


(array([1.31398405, 0.72012987, 1.26451081]),
 1.0,
 array([1.33823269, 1.30039044, 1.28197304, 1.27300949]),
 array([0.86282984, 0.71712771, 1.4574832 ]),
 1.0,
 array([1.50647764, 1.48139995, 1.46915822, 1.46318239]))

In [30]:
# fit_exp_offset(all_entropies_unet[1], "for_unet_entropies_postsism", num_data_point=3)

##### For Latex results table: SAS performance

In [31]:
# Dice 
presas_result_str = ""
for score in presas_dice_score_list:
    presas_result_str += f"& {score:.4f}"
    
postsas_result_str = ""
for score in postsas_dice_score_list:
    postsas_result_str += f"& {score:.4f}"
    
print(f"Pre-SAS dice: {presas_result_str}")
print(f"Post-SAS dice: {postsas_result_str}")

# mIoU
presas_result_str = ""
for score in presas_miou_score_list:
    presas_result_str += f"& {score:.4f}"
    
postsas_result_str = ""
for score in postsas_miou_score_list:
    postsas_result_str += f"& {score:.4f}"
    


print(f"Pre-SAS miou: {presas_result_str}")
print(f"Post-SAS miou: {postsas_result_str}")

Pre-SAS dice: & 0.7244& 0.7843& 0.8116& 0.8254& 0.8320& 0.8431& 0.8541
Post-SAS dice: & 0.7266& 0.8030& 0.8242& 0.8089& 0.8342& 0.8438& 0.8535
Pre-SAS miou: & 0.5733& 0.6513& 0.6874& 0.7061& 0.7156& 0.7316& 0.7485
Post-SAS miou: & 0.5756& 0.6758& 0.7050& 0.6838& 0.7187& 0.7324& 0.7470


##### For Latex results table: Mask Quality

In [32]:
categories = ['presas', 'postsas']
percentages = ['10%', '20%', '30%', '40%', '50%', '60%', '70%']

unusables, unreliables, moderates, goods = "", "", "", ""

for category in categories:
    unusables_row, unreliables_row, moderates_row, goods_row = "", "", "", ""
    
    for percent in percentages:
        scores = sorted(dice_scores[category][percent])
        
        unusable = sum(1 for score in scores if score == 0.0)
        unreliable = sum(1 for score in scores if 0.0 < score <= 0.5)
        moderate = sum(1 for score in scores if 0.5 < score <= 0.8)
        good = sum(1 for score in scores if score > 0.8)
        
        unusables_row += f"& {unusable/len(scores)*100:.2f}"
        unreliables_row += f"& {unreliable/len(scores)*100:.2f}"
        moderates_row += f"& {moderate/len(scores)*100:.2f}"
        goods_row += f"& {good/len(scores)*100:.2f}"
    

    print(category)
    print("="*80)
    print("Unusables:\t", unusables_row)
    print("Unreliables:\t", unreliables_row)
    print("Moderates:\t", moderates_row)
    print("Goods:\t\t", goods_row)
    print()


presas
Unusables:	 & 1.00& 0.00& 0.50& 0.50& 0.00& 0.00& 0.00
Unreliables:	 & 13.50& 13.50& 9.50& 8.50& 8.00& 7.50& 6.50
Moderates:	 & 28.50& 24.50& 20.00& 22.00& 22.50& 23.00& 19.50
Goods:		 & 57.00& 62.00& 70.00& 69.00& 69.50& 69.50& 74.00

postsas
Unusables:	 & 0.50& 0.50& 0.50& 0.50& 0.50& 0.00& 0.50
Unreliables:	 & 18.00& 10.50& 7.00& 10.00& 7.50& 7.50& 7.00
Moderates:	 & 24.00& 28.50& 21.00& 19.00& 21.00& 23.00& 19.00
Goods:		 & 57.50& 60.50& 71.50& 70.50& 71.00& 69.50& 73.50



In [33]:
# ## Splitwise-Modelwise Mask Comparison
# full_dataset = get_dataset(image_size=(256, 256), mask_size=(256, 256))
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# data = random.choice(full_dataset)

# results = get_sas_modelwise_results(data=data,
#                                     model_dir=f"{unet_semi_auto_path}/models", 
#                                     model_name="unet", 
#                                     model_config="114", 
#                                     device=device)

In [34]:
# plot_modelwise_comparison(image_data=data, results=results, savefig_name=f"{save_folder}/unet_modelwise_mask_quality_viz.pdf")